In [ ]:
#Directory
dir = "/content/drive/MyDrive/Personal/Apziva/images"
# Setting random state for consistency
np.random.seed(123)
df = pd.read_csv(f"{dir}/dementia_patients_health_data.csv")

/usr/local/lib/python3.12/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU
  warnings.warn(
Downloading...
From: https://docs.google.com/spreadsheets/d/117X6i53dKiO7w6kuA1g1TpdTlv1173h_dPlJt5cNNMU/edit?usp=sharing
To: /content/edit?usp=sharing
252kB [00:00, 1.31MB/s]


In [ ]:
img_dim = 256

In [ ]:
import os, sys, random, warnings
from pathlib import Path
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm

# Params
IMG_WIDTH  = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_NOT   = f"{dir}/training/notflip"
TRAIN_FLIP  = f"{dir}/training/flip"
TEST_NOT    = f"{dir}/testing/notflip"
TEST_FLIP   = f"{dir}/testing/flip"

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 0
random.seed(seed)
np.random.seed(seed)

def _list_images(p):
    p = Path(p)
    exts = ("*.jpg","*.jpeg","*.png","*.bmp","*.tif","*.tiff")
    files = []
    for e in exts:
        files += list(p.rglob(e))   # works whether images are directly inside or nested per-id
    return sorted(files)

def _load_and_resize(path):
    img = imread(str(path))[:, :, :IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode="constant", preserve_range=True, anti_aliasing=True)
    return img.astype(np.uint8)

def load_binary_split():
    # Collect file paths and labels
    train_paths = _list_images(TRAIN_NOT)
    train_labels = [0]*len(train_paths)
    fp = _list_images(TRAIN_FLIP)
    train_paths += fp
    train_labels += [1]*len(fp)

    test_paths  = _list_images(TEST_NOT)
    test_labels = [0]*len(test_paths)
    fp = _list_images(TEST_FLIP)
    test_paths += fp
    test_labels += [1]*len(fp)

    # Shuffle deterministically
    rng = np.random.default_rng(seed)
    idx = rng.permutation(len(train_paths))
    train_paths = [train_paths[i] for i in idx]
    train_labels = np.array([train_labels[i] for i in idx], dtype=np.int32)

    idx = rng.permutation(len(test_paths))
    test_paths = [test_paths[i] for i in idx]
    test_labels = np.array([test_labels[i] for i in idx], dtype=np.int32)

    # Load pixels
    print("Loading and resizing training images...")
    X_train = np.zeros((len(train_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    for i, p in tqdm(list(enumerate(train_paths)), total=len(train_paths)):
        X_train[i] = _load_and_resize(p)

    print("Loading and resizing test images...")
    X_test = np.zeros((len(test_paths), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
    for i, p in tqdm(list(enumerate(test_paths)), total=len(test_paths)):
        X_test[i] = _load_and_resize(p)

    # Scale to [0,1] floats for CNN input
    X_train = (X_train / 255.0).astype(np.float32)
    X_test  = (X_test  / 255.0).astype(np.float32)

    return X_train, train_labels, X_test, test_labels, train_paths, test_paths

X_train, y_train, X_test, y_test, train_files, test_files = load_binary_split()
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


In [ ]:

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
model = Sequential([
    Input(shape = (img_dim, img_dim, 3)),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    Conv2D(8, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    Conv2D(16, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Flatten(),
    Dense(128, activation = 'relu'),
    Dense(4, activation = 'softmax')
])
